In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fake_useragent
import re
import unicodedata
import warnings
warnings.filterwarnings("ignore")

ua = fake_useragent.UserAgent()

In [2]:
def get_url(category, tot_pages):
    for page in range(0,tot_pages+1):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')       

        url_main = f"https://petrasport.ru/{category}/p/{page}/"
        response = requests.get(
            url_main,
            headers={"user-agent": ua.random},
            verify=False,
            allow_redirects=False,
        )
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = 'product-name')
        for item in data:
            url = item.find('a', href = True)
            yield ('https://petrasport.ru' + url['href'])

In [3]:
def get_item(gen_category, category, tot_pages):
    count = 0
    for url_item in get_url(category, tot_pages):
        count += 1
        response = requests.get(url_item, headers = {'user-agent': ua.random}, verify=False, allow_redirects=False)
        soup = BeautifulSoup(response.content, 'lxml')

        try:
            title = soup.find('h1').text
            title = title.replace('\r', '').replace('\t', '').replace('\n', '').strip()
        except:
            title = ''

        article = ''

        try:
            image = soup.find(class_ = 'product-image')
            url = 'https://petrasport.ru' + image.find('a')['href']
            image_ref = f"images/{gen_category}/{count}_petrasport_{article}.jpeg"
            with open(f'images/{gen_category}/{count}_petrasport_{article}.jpeg', 'wb') as f:
                r = requests.get(url)
                f.write(r.content)     


        except:
            print(f'Log: no image for {article} found...')
            url = ''
            image_ref = ''

        try:
            char = soup.find_all(class_ = 'even')
            char_string = ''
            for ch in char:
                children = ch.children
                for child in children:
                    if bool(child.find(class_ = 'popup-param')):
                        pass
                    else:
                        char_string += child.text
                        char_string += ' '
            
            char = soup.find_all(class_ = 'add')
            for ch in char:
                children = ch.children
                for child in children:
                    if bool(child.find(class_ = 'popup-param')):
                        pass
                    else:
                        char_string += child.text
                        char_string += ' '
            chars = unicodedata.normalize('NFKD',char_string).strip()
        except:
            chars = ''


        desc = ''

        try:
            price = soup.find(class_ = 'price-current').text
            price = unicodedata.normalize('NFKD',price).replace(' ', '').replace('руб.', '')
        except:
            price =''

        try:
            cat = soup.find(class_ = 'site-path').text
        except:
            cat = []
        try:
            cat_1 = cat.split('/')[0].strip()
        except:
            cat_1 = ''
        try:
            cat_2 = cat.split('/')[1].strip()
        except:
            cat_2 = ''
        try:
            cat_3 = cat.split('/')[2].strip()
        except:
            cat_3 = ''

        yield  title, article, price, cat_1, cat_2, cat_3, url, desc, chars, image_ref

In [4]:
df_columns = ['title', 'article','price', 'cat_1', 'cat_2', 'cat_3','url', 
              'description', 'caracteristics', 'img_ref']

df = pd.DataFrame(columns = df_columns)
save_frequency = 100
log_frequency = 10

In [5]:
import time

def to_csv(gen_category, category, tot_pages):
    counter = 0  

    for item in get_item(gen_category, category, tot_pages):
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
            time.sleep(0.1)

            counter += 1
            if counter % save_frequency == 0:
                df.to_csv(f'{gen_category}_petrasport.csv')
                print(f'{counter} items saved in csv..')
    df.to_csv(f'{gen_category}_petrasport.csv')

    print(f'\nTotal items saved: {counter}')  

In [6]:
to_csv("trenazhery_na_svobodnykh_vesakh", "trenazhery-na-svobodnyh-vesah", 8)

Log: working over page 0...
100 items saved in csv..
Log: no image for  found...
Log: no image for  found...

Total items saved: 164
